In [1]:
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path
import pynmea2
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from tqdm.notebook import tqdm
import copy

In [2]:
files_list = glob(r'../../Data/Caravela/CaravelaPCS/Datalog/Y20/*/*'+ '/*recordDataPort_D*')  
files_list.sort()
li = []

for filename in tqdm(range(4,len(files_list))): # take files from Caravelas launch onwards
    with open(files_list[filename],'r') as myfile:
        content = myfile.readlines()[1:-1] #skip first line of file as this is likey to have not been written correctly
    frame = pd.DataFrame(data = np.array(content), columns= ['original'])
    li.append(frame)
df = pd.concat(li, axis=0,ignore_index=True)

In [3]:
del frame, content, li # clear the things we dont need open
myfile.close()

In [4]:
df['nmea_ident'] = copy.deepcopy(df.original.str[16:22]) #create a new column of the nmea string code
df['desired'] = False #create column to decide which rows to keep
df.loc[df['nmea_ident'] == '$WIMDA', 'desired'] = True #set the rows with the desired nmea string identity to true
des = df[df.desired] # create new dataframe with desired rows

In [5]:
des = des.reset_index() #tidy up the new dataframe
des = des.drop(['index'], axis=1)

In [6]:
strings = [] # create empty list to hold nmea strings 
des['NMEA'] = '' #create new column in dataframe
for i in tqdm(range(len(des['original']))): 
    try: 
        nmea = pynmea2.parse(des['original'][i][16:], check=True) #parse nmea string and check the checksum is correct
    except ValueError:
        nmea = np.nan # replace errors with nan
    strings.append(nmea)
des['NMEA'] = strings #populate the dataframe

In [7]:
des

,original,nmea_ident,desired,NMEA
0,"200122,12:04:48,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B"
1,"200122,12:04:49,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B"
2,"200122,12:04:50,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B"
3,"200122,12:04:51,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B"
4,"200122,12:04:52,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B"
...,...,...,...,...
2837411,"200224,14:20:18,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A"
2837412,"200224,14:20:19,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A"
2837413,"200224,14:20:20,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A"
2837414,"200224,14:20:21,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A"


In [8]:
parsed = [] #create emptry list for parsed datetimes
des['datetime'] = ''
for i in tqdm(range(len(des['original']))):
    try: 
        time_in = ((datetime.strptime(des.original[i][:15], '%y%m%d,%H:%M:%S')))
    except ValueError:
        time_in = np.nan
    parsed.append(time_in)
des['datetime'] = parsed

In [56]:
des['pressure[bar]'] = ''#create empty column for pressure data
a=[] 
for i in tqdm(range(len(des['NMEA']))): #extract pressure from parsed nmea string
    if type(des['NMEA'][i]) != pynmea2.types.talker.MDA:
        a.append(np.nan)
    else:
        a.append(des['NMEA'][i].b_pressure_bar)
des['pressure[bar]'] = a

In [96]:
def variable_extract(column,variable):
    a=[] #create empty list to put extracted variable int
    for i in range(0,10): 
        if type(column[i]) != pynmea2.types.talker.MDA: #if cell is not expected pynmea2 parsed type, add nan
            a.append(np.nan)
    else:
        a.append(variable) #otherwise, extract desired variable
    return a

In [97]:

variable_extract(des['NMEA'], des.NMEA[i].b_pressure_bar)
des['test'] = a

In [105]:
des

,original,nmea_ident,desired,NMEA,datetime,pressure[bar],test
0,"200122,12:04:48,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B",2020-01-22 12:04:48,1.0142,1.0142
1,"200122,12:04:49,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B",2020-01-22 12:04:49,1.0142,1.0142
2,"200122,12:04:50,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B",2020-01-22 12:04:50,1.0142,1.0142
3,"200122,12:04:51,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B",2020-01-22 12:04:51,1.0142,1.0142
4,"200122,12:04:52,$WIMDA,29.9493,I,1.0142,B,23.0...",$WIMDA,True,"$WIMDA,29.9493,I,1.0142,B,23.0,C,,,,,,,,,,,,,,*3B",2020-01-22 12:04:52,1.0142,1.0142
...,...,...,...,...,...,...,...
2837411,"200224,14:20:18,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A",2020-02-24 14:20:18,1.0173,1.0173
2837412,"200224,14:20:19,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A",2020-02-24 14:20:19,1.0173,1.0173
2837413,"200224,14:20:20,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A",2020-02-24 14:20:20,1.0173,1.0173
2837414,"200224,14:20:21,$WIMDA,30.0409,I,1.0173,B,25.7...",$WIMDA,True,"$WIMDA,30.0409,I,1.0173,B,25.7,C,,,,,,,,,,,,,,*3A",2020-02-24 14:20:21,1.0173,1.0173


In [88]:
des.NMEA[0]

<MDA(b_pressure_inch=Decimal('29.9493'), inches='I', b_pressure_bar=Decimal('1.0142'), bars='B', air_temp=Decimal('23.0'), a_celsius='C', water_temp=None, w_celsius='', rel_humidity=None, abs_humidity=None, dew_point=None, d_celsius='', direction_true=None, true='', direction_magnetic=None, magnetic='', wind_speed_knots=None, knots='', wind_speed_meters=None, meters='')>

In [112]:
b = np.where(des['pressure[bar]'] == des.test, 'True', 'False')

In [113]:
np.size(b) - np.count_nonzero(b) # array size - nonfalse size to check all values are true. true represents test function= explicit for loop

0